In [1]:
import zipfile
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"
zip = zipfile.ZipFile("nlp_getting_started.zip")
zip.extractall()
zip.close()

--2024-07-11 05:19:42--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.207, 74.125.20.207, 108.177.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2024-07-11 05:19:42 (80.7 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import sklearn

tf.random.set_seed(42)

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [4]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
import random

rnd = random.randint(0,len(train_df)-5)

for row in train_df[['text', 'target']][rnd:rnd+5].itertuples():
  _,text,target=row
  print(f"text: {text}")
  print(f"target: {target}")

text: Several wild fires have burned a lot of land in California. Here is one of the larger fires. http://t.co/2M1gNeaiFl http://t.co/UQh85MiP0v
target: 1
text: Map shows where all of California's wild#fires are burning: This map created by CAL FIREÛ_ http://t.co/0x8jAQToWM http://t.co/m1RoSi2Wcs
target: 1
text: Some great footage of STRONG work from San Bernardino County Fire who is also working the wild fires right now.... http://t.co/QCYQHvn2Ha
target: 1
text: God forbid this is true #California has enough problems with severe #drought &amp; #wild fires. http://t.co/CMsgexM4FC #Nuclear Power #SanOnofre
target: 1
text: Firefighters Headed To California To Fight Wild Fires http://t.co/J2PYkYo0EN
target: 1


In [6]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_targets, val_targets = train_test_split(train_df['text'],
                                                                              train_df['target'],
                                                                              test_size=0.2,
                                                                              random_state=42)

In [7]:
def evaluate_model(model, val_sentences, val_targets):
  model_accuracy = sklearn.metrics.accuracy_score(val_targets, model.predict(val_sentences))
  model_f1score = sklearn.metrics.f1_score(val_targets, model.predict(val_sentences))
  model_recall = sklearn.metrics.recall_score(val_targets, model.predict(val_sentences))
  model_results = {"accuracy" : model_accuracy,
                   "f1score" : model_f1score,
                   "recall" : model_recall}

  return model_results

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # converts text to numbers (Tokenisation)
    ("clf", MultinomialNB()) #Models data
])

model0.fit(train_sentences, train_targets)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [9]:
evaluate_model(model0, val_sentences, val_targets)

{'accuracy': 0.7997373604727511,
 'f1score': 0.7284060552092609,
 'recall': 0.6302003081664098}

In [10]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens = 10000,
    output_sequence_length=15,
    standardize = "lower_and_strip_punctuation",
    split = 'whitespace',
    ngrams = None,
    output_mode = 'int',
    pad_to_max_tokens = False
)

embedding = tf.keras.layers.Embedding(input_dim = 10000,
                                      output_dim = 15,
                                      embeddings_initializer = 'uniform',
                                      input_length = 15,
                                      name = 'Embedding_1')

In [11]:
text_vectorizer.adapt(train_sentences)

In [14]:
inputs_1 = tf.keras.layers.Input(shape=(1,), dtype = tf.string)
x_1 = text_vectorizer(inputs_1)
x_1 = embedding(x_1)
x_1 = tf.keras.layers.GlobalAveragePooling1D()(x_1)
outputs_1 = tf.keras.layers.Dense(1, activation = 'sigmoid')(x_1)
model_1 = tf.keras.Model(inputs_1, outputs_1)

model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                 optimizer=tf.keras.optimizers.Adam(),
                 metrics=['accuracy'])

In [15]:
model_1.fit(train_sentences,
            train_targets,
            epochs=5,
            validation_data=(val_sentences, val_targets))

Epoch 1/5
191/191 [==============================] - 2s 5ms/step - loss: 0.6484 - accuracy: 0.7314 - val_loss: 0.6230 - val_accuracy: 0.7045
Epoch 2/5
191/191 [==============================] - 1s 5ms/step - loss: 0.5748 - accuracy: 0.7632 - val_loss: 0.5710 - val_accuracy: 0.7597
Epoch 3/5
191/191 [==============================] - 1s 5ms/step - loss: 0.5096 - accuracy: 0.8067 - val_loss: 0.5272 - val_accuracy: 0.7912
Epoch 4/5
191/191 [==============================] - 1s 5ms/step - loss: 0.4538 - accuracy: 0.8345 - val_loss: 0.4955 - val_accuracy: 0.7991
Epoch 5/5
191/191 [==============================] - 1s 5ms/step - loss: 0.4094 - accuracy: 0.8511 - val_loss: 0.4746 - val_accuracy: 0.8063
